# Parameter estimation for a linear operator with pyGPs

Assumptions: <br>
$\mathcal{L}_x^{\phi}u(x) = f(x)$ <br>
$u(x) \sim \mathcal{GP}(0, k_{uu}(x, x', \theta))$ <br>
$f(x) \sim \mathcal{GP}(0, k_{ff}(x, x', \theta, \phi))$ <br>
$\theta = \{\sigma, l\}$ <br>

<br>
Problem at hand: Given $\{X_u, y_u\}$ and $\{X_f, y_f\}$, estimate $\phi$. <br>
<br>
We employ a GP with a RBF kernel for u and f: <br>
$k_{uu}(x_i, x_j; \theta_u) = \sigma_u^2 \exp(-\frac{1}{2l_u^2}(x_i - x_j)^2) \\
k_{ff}(x_i, x_j; \theta_f) = \sigma_f^2 \exp(-\frac{1}{2l_f^2}(x_i - x_j)^2) $ <br> <br>

We use the known transformation behavior of Gaussian Processes: <br>
$k_{ff}(x_i, x_j; \theta, \phi) \\
= \mathcal{L}_{x_i}^{\phi}\mathcal{L}_{x_j}^{\phi}k_{uu}(x_i, x_j; \theta) \\
= \phi^2\sigma_u^2 \exp(-\frac{1}{2l_u^2}(x_i - x_j)^2)$ <br> <br>

Looking at an entry of the covariance matrix, where $x_i = x_j$ (a diagonal element), it follows that
$\sigma_f^2 = \phi^2\sigma_u^2$, i.e. <br>
$\phi = \frac{\sigma_f}{\sigma_u}$.




In [ ]:
import numpy as np
import pyGPs

# Linear functional L = phi*u(x) was chosen. Desired result: phi = 12.0.

# Generating data
t = 1
u0 = 5
dt = 0.1
#u_{n} = L(u_{n+1})
#L = I - dt*mu*d_xx
x_u = np.linspace(0,2*np.pi,20)
y_u = (4*u0/np.pi)*np.sin(np.pi*x_u)*np.exp(-np.square(np.pi)*t)
x_f = np.linspace(0,2*np.pi,20)
y_f = 0.1*(4*u0/np.pi)*np.sin(np.pi*x_f)*np.exp(-np.square(np.pi)*t)+dt*(4*u0*np.pi)*np.sin(np.pi*x_f)*np.exp(-np.square(np.pi)*t)     # You can vary the factor, and that very factor should be the output of this program

# The function u is assumed to be a Gaussian Process. 
# After a linear transformation, f has to be a Gaussian Process as well.

model_u = pyGPs.GPR()
model_u.setData(x_u, y_u)
model_u.optimize(x_u, y_u)

model_f = pyGPs.GPR()
model_f.setData(x_f, y_f)
model_f.optimize(x_f, y_f)


# Note that in hyp only the logarithm of the hyperparameter is stored!
# Characteristic length-scale is equal to np.exp(hyp[0]) (Default: 1)
# Signal variance is equal to np.exp(hyp[1]) (Default: 1)

print(np.exp(model_f.covfunc.hyp[1])/np.exp(model_u.covfunc.hyp[1]))	# This should give 12 as output

# My Output: 12.0486915165

Number of line searches 21
Number of line searches 10


474983.1781946725
